# New York City Taxi Fare Prediction

## Description
В этом конкурсе playground, проводимом в партнерстве с Google Cloud и Coursera, вам предлагается спрогнозировать стоимость проезда (включая дорожные сборы) на такси в Нью-Йорке с учетом мест посадки и высадки пассажиров. Хотя вы можете получить базовую оценку, основанную только на расстоянии между двумя точками, это приведет к тому, что RMSE составит от 5 до 8 долларов, в зависимости от используемой модели (пример такого подхода в Kernels приведен в исходном коде). Ваша задача - добиться большего успеха, используя методы машинного обучения!

Чтобы научиться легко обрабатывать большие массивы данных и решить эту проблему с помощью TensorFlow, рассмотрите возможность прохождения курса машинного обучения с помощью TensorFlow на облачной платформе Google на Coursera. Проблема оплаты проезда на такси - одна из нескольких реальных проблем, которые используются в качестве примеров в серии курсов. Чтобы упростить задачу, перейдите по ссылке Coursera.org/NEXTextended и получите эту специализацию бесплатно в течение первого месяца!


## Dataset Description

### File descriptions
- <b>train.csv</b> - Ввод параметров и целевых значений fare_amount для обучающего набора (около 55 миллионов строк).
- <b>test.csv</b> - Ввод параметров для тестового набора (около 10 тысяч строк). Ваша цель - предсказать стоимость проезда для каждой строки.
- <b>sample_submission.csv</b> - файл с образцом отправки в правильном формате (ключ столбца и стоимость проезда). Этот файл "предсказывает", что стоимость проезда составит $11,35 для всех строк, что является средним значением стоимости проезда из обучающего набора.

### Data fields
#### ID
- <b>key</b> - уникальная `string`, идентифицирующая каждую строку как в обучающем, так и в тестовом наборах. Состоит из <b>pickup_datetime</b> и уникального целого числа, но это не имеет значения, его следует использовать просто как поле уникального идентификатора. Требуется для отправки в формате CSV. Не обязательно требуется в обучающем наборе, но может быть полезно для моделирования "файла отправки" при выполнении перекрестной проверки в обучающем наборе.

### Features
- <b>pickup_datetime</b> - `timestamp` значение временной метки, указывающее, когда началась поездка на такси.
- <b>pickup_longitude</b> - `float` значение с плавающей точкой для координаты долготы того места, где началась поездка на такси.
- <b>pickup_latitude</b> - `float` значение с плавающей точкой для координаты широты того места, где началась поездка на такси.
- <b>dropoff_longitude</b> - `float` значение с плавающей точкой для координаты долготы того места, где закончилась поездка на такси.
- <b>dropoff_latitude</b> - `float` значение с плавающей точкой для координаты широты того места, где закончилась поездка на такси.
- <b>passenger_count</b> - целое число, указывающее количество пассажиров в поездке на такси.

### Target
- <b>fare_amount</b> - `float` плавающая сумма стоимости поездки на такси в долларах США. Это значение есть только в обучающем наборе; это то, что вы прогнозируете в тестовом наборе, и оно требуется в вашем CSV-файле для отправки.

# Code

In [32]:
import pandas as pd

train_path = 'data/new-york-city-taxi-fare-prediction/train.csv'
test_path = 'data/new-york-city-taxi-fare-prediction/test.csv'

train_data = pd.read_csv(train_path, nrows=100000)

train_data.head(5)

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1


In [33]:
train_data = train_data.dropna(
    subset=['pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude']
)


## Извлечение фич из времени начала поездки

In [34]:
from datetime import datetime

train_data['pickup_datetime'] = pd.to_datetime(train_data['pickup_datetime'], errors='coerce')

train_data['hour'] = train_data['pickup_datetime'].dt.hour
train_data['minute'] = train_data['pickup_datetime'].dt.minute
train_data['day_of_week'] = train_data['pickup_datetime'].dt.weekday
train_data['day_of_month'] = train_data['pickup_datetime'].dt.day
train_data['month'] = train_data['pickup_datetime'].dt.month
train_data['year'] = train_data['pickup_datetime'].dt.year

train_data['is_weekend'] = (train_data['day_of_week'] >= 5).astype(int)
train_data['is_night_trip'] = ((train_data['hour'] >= 23) | (train_data['hour'] <= 5)).astype(int)
train_data['is_rush_hour'] = ((train_data['hour'] >= 7) & (train_data['hour'] <= 9)) | ((train_data['hour'] >= 16) & (train_data['hour'] <= 19))
train_data['season'] = train_data['month'] % 12 // 3 + 1

train_data.head(5)

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,hour,minute,day_of_week,day_of_month,month,year,is_weekend,is_night_trip,is_rush_hour,season
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21+00:00,-73.844311,40.721319,-73.841610,40.712278,1,17,26,0,15,6,2009,0,0,True,3
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16+00:00,-74.016048,40.711303,-73.979268,40.782004,1,16,52,1,5,1,2010,0,0,True,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00+00:00,-73.982738,40.761270,-73.991242,40.750562,2,0,35,3,18,8,2011,0,1,False,3
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42+00:00,-73.987130,40.733143,-73.991567,40.758092,1,4,30,5,21,4,2012,1,1,False,2
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00+00:00,-73.968095,40.768008,-73.956655,40.783762,1,7,51,1,9,3,2010,0,0,True,2


## Извчеление географических признаков

In [35]:
from geopy.distance import geodesic
import numpy as np

print("Начало 'Извчеление географических признаков'")

# Очистка данных (удаляем строки с некорректными координатами)
print("Очистка данных (удаляем строки с некорректными координатами)...")
train_data = train_data[
    (train_data['pickup_latitude'].between(-90, 90)) & 
    (train_data['dropoff_latitude'].between(-90, 90)) & 
    (train_data['pickup_longitude'].between(-180, 180)) & 
    (train_data['dropoff_longitude'].between(-180, 180))
].dropna(subset=['pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude'])

# Преобразуем координаты в float (если они строковые)
print("Преобразуем координаты в float (если они строковые)...")
train_data[['pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude']] = \
    train_data[['pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude']].astype(float)

# Функция Haversine distance (расстояние по сфере Земли)
print("Функция Haversine distance (расстояние по сфере Земли)...")
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Радиус Земли в км
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    return 2 * R * np.arcsin(np.sqrt(a))

# Добавляем расстояние по Haversine
print("Добавляем расстояние по Haversine...")
train_data['haversine_distance'] = haversine_distance(
    train_data['pickup_latitude'], train_data['pickup_longitude'],
    train_data['dropoff_latitude'], train_data['dropoff_longitude']
)

# Manhattan distance (по прямоугольной сетке)
print("Manhattan distance (по прямоугольной сетке)...")
train_data['manhattan_distance'] = (
    abs(train_data['pickup_latitude'] - train_data['dropoff_latitude']) +
    abs(train_data['pickup_longitude'] - train_data['dropoff_longitude'])
) * 111  # Один градус ≈ 111 км

# Признак направления движения (bearing)
def bearing(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    x = np.sin(dlon) * np.cos(lat2)
    y = np.cos(lat1) * np.sin(lat2) - np.sin(lat1) * np.cos(lat2) * np.cos(dlon)
    return np.degrees(np.arctan2(x, y))

train_data['bearing'] = bearing(
    train_data['pickup_latitude'], train_data['pickup_longitude'],
    train_data['dropoff_latitude'], train_data['dropoff_longitude']
)

# Координаты аэропортов и центра города
JFK = (40.6413, -73.7781)
LGA = (40.7769, -73.8740)
EWR = (40.6895, -74.1745)
DOWNTOWN = (40.7580, -73.9855)

# Функция для проверки близости к заданной точке (по расстоянию)
def is_near_location(lat, lon, location, threshold=1):
    return geodesic((lat, lon), location).km < threshold

# Признаки близости к аэропортам
print("Признаки близости к аэропортам...")
train_data['pickup_near_airport'] = train_data.apply(
    lambda row: int(
        is_near_location(row['pickup_latitude'], row['pickup_longitude'], JFK) or
        is_near_location(row['pickup_latitude'], row['pickup_longitude'], LGA) or
        is_near_location(row['pickup_latitude'], row['pickup_longitude'], EWR)
    ), axis=1
)

train_data['dropoff_near_airport'] = train_data.apply(
    lambda row: int(
        is_near_location(row['dropoff_latitude'], row['dropoff_longitude'], JFK) or
        is_near_location(row['dropoff_latitude'], row['dropoff_longitude'], LGA) or
        is_near_location(row['dropoff_latitude'], row['dropoff_longitude'], EWR)
    ), axis=1
)

# Признаки близости к центру Манхэттена
print("Признаки близости к центру Манхэттена...")
train_data['pickup_near_downtown'] = train_data.apply(
    lambda row: int(is_near_location(row['pickup_latitude'], row['pickup_longitude'], DOWNTOWN)), axis=1
)

train_data['dropoff_near_downtown'] = train_data.apply(
    lambda row: int(is_near_location(row['dropoff_latitude'], row['dropoff_longitude'], DOWNTOWN)), axis=1
)

print("Проверяем, что признаки добавлены...")
train_data.head(5)  # Проверяем, что признаки добавлены


Начало 'Извчеление географических признаков'
Очистка данных (удаляем строки с некорректными координатами)...
Преобразуем координаты в float (если они строковые)...
Функция Haversine distance (расстояние по сфере Земли)...
Добавляем расстояние по Haversine...
Manhattan distance (по прямоугольной сетке)...
Признаки близости к аэропортам...
Признаки близости к центру Манхэттена...
Проверяем, что признаки добавлены...
                             key  fare_amount           pickup_datetime  \
0    2009-06-15 17:26:21.0000001          4.5 2009-06-15 17:26:21+00:00   
1    2010-01-05 16:52:16.0000002         16.9 2010-01-05 16:52:16+00:00   
2   2011-08-18 00:35:00.00000049          5.7 2011-08-18 00:35:00+00:00   
3    2012-04-21 04:30:42.0000001          7.7 2012-04-21 04:30:42+00:00   
4  2010-03-09 07:51:00.000000135          5.3 2010-03-09 07:51:00+00:00   

   pickup_longitude  pickup_latitude  dropoff_longitude  dropoff_latitude  \
0        -73.844311        40.721319         -73.84161

In [36]:
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split

# Выбираем признаки
features = [
    'hour', 'day_of_week', 'month', 'haversine_distance', 'manhattan_distance',
    'pickup_near_airport', 'dropoff_near_airport', 'is_weekend', 'is_rush_hour'
]

X = train_data[features]
y = train_data['fare_amount']

# Разделяем на train и test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Обучаем CatBoost
model = CatBoostRegressor(iterations=500, depth=6, learning_rate=0.1, loss_function='RMSE', verbose=50)
model.fit(X_train, y_train)

# Оцениваем модель
y_pred = model.predict(X_test)

from sklearn.metrics import mean_squared_error
print("RMSE:", mean_squared_error(y_test, y_pred, squared=False))


0:	learn: 9.1066652	total: 55.6ms	remaining: 27.8s
50:	learn: 5.1038674	total: 352ms	remaining: 3.1s
100:	learn: 5.0052434	total: 630ms	remaining: 2.49s
150:	learn: 4.9323446	total: 998ms	remaining: 2.31s
200:	learn: 4.8803713	total: 1.36s	remaining: 2.03s
250:	learn: 4.8332199	total: 1.8s	remaining: 1.78s
300:	learn: 4.7956430	total: 2.07s	remaining: 1.37s
350:	learn: 4.7601694	total: 2.31s	remaining: 979ms
400:	learn: 4.7330006	total: 2.57s	remaining: 635ms
450:	learn: 4.7046526	total: 3s	remaining: 326ms
499:	learn: 4.6799888	total: 3.28s	remaining: 0us
RMSE: 4.87214766473839


/home/repos/jupiter/venv/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [ ]:
# from geopy.distance import geodesic
# import numpy as np

# # Функция для расчета Haversine distance
# def haversine_distance(lat1, lon1, lat2, lon2):
#     R = 6371  # Радиус Земли в километрах
#     lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
#     dlat = lat2 - lat1
#     dlon = lon2 - lon1
#     a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
#     return 2 * R * np.arcsin(np.sqrt(a))

# train_data['haversine_distance'] = train_data.apply(
#     lambda row: haversine_distance(
#         row['pickup_latitude'], 
#         row['pickup_longitude'],
#         row['dropoff_latitude'], 
#         row['dropoff_longitude']
#     ), 
#     axis=1
# )

# # Manhattan distance
# train_data['manhattan_distance'] = (
#     abs(train_data['pickup_latitude'] - train_data['dropoff_latitude']) +
#     abs(train_data['pickup_longitude'] - train_data['dropoff_longitude'])
# ) * 111  # градус ≈ 111 км

# # Признак направления движения (bearing)
# def bearing(lat1, lon1, lat2, lon2):
#     lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
#     dlon = lon2 - lon1
#     x = np.sin(dlon) * np.cos(lat2)
#     y = np.cos(lat1) * np.sin(lat2) - np.sin(lat1) * np.cos(lat2) * np.cos(dlon)
#     return np.degrees(np.arctan2(x, y))

# train_data['bearing'] = train_data.apply(
#     lambda row: bearing(
#         row['pickup_latitude'], 
#         row['pickup_longitude'],
#         row['dropoff_latitude'], 
#         row['dropoff_longitude']
#     ), 
#     axis=1
# )

# # Координаты аэропортов
# JFK = (40.6413, -73.7781)
# LGA = (40.7769, -73.8740)
# EWR = (40.6895, -74.1745)
# DOWNTOWN = (40.7580, -73.9855)

# # Функция для проверки, находится ли точка рядом с аэропортом (1 км)
# def is_near_location(lat, lon, location, threshold=1):
#     return geodesic((lat, lon), location).km < threshold

# train_data['pickup_near_airport'] = train_data.apply(
#     lambda row: 
#         is_near_location(row['pickup_latitude'], row['pickup_longitude'], JFK) |
#         is_near_location(row['pickup_latitude'], row['pickup_longitude'], LGA) |
#         is_near_location(row['pickup_latitude'], row['pickup_longitude'], EWR), 
#         axis=1
# )

# train_data['dropoff_near_airport'] = train_data.apply(
#     lambda row: 
#         is_near_location(row['dropoff_latitude'], row['dropoff_longitude'], JFK) |
#         is_near_location(row['dropoff_latitude'], row['dropoff_longitude'], LGA) |
#         is_near_location(row['dropoff_latitude'], row['dropoff_longitude'], EWR), 
#         axis=1
# )

# train_data['pickup_near_downtown'] = train_data.apply(
#     lambda row: 
#         is_near_location(row['pickup_latitude'], row['pickup_longitude'], DOWNTOWN), 
#         axis=1
# )
# train_data['dropoff_near_downtown'] = train_data.apply(
#     lambda row: 
#         is_near_location(row['dropoff_latitude'], row['dropoff_longitude'], DOWNTOWN), 
#         axis=1
# )


/home/repos/jupiter/venv/lib/python3.12/site-packages/geopy/point.py:472: UserWarning: Latitude normalization has been prohibited in the newer versions of geopy, because the normalized value happened to be on a different pole, which is probably not what was meant. If you pass coordinates as positional args, please make sure that the order is (latitude, longitude) or (y, x) in Cartesian terms.
  return cls(*args)


ValueError: Latitude must be in the [-90; 90] range.